In [ ]:
import numpy as np
import pylab as plt
import svgwrite
from skimage import io, filters, measure, feature, exposure, morphology, restoration, util
from scipy import ndimage, spatial
from scipy.ndimage import rotate
import os
import random
from IPython.display import SVG
%matplotlib inline

In [ ]:
toDraw = 'MobiusAlpha/'
ao_monkey = io.imread(toDraw + 'AO.png')
uv_monkey = io.imread(toDraw + 'UV.png')
normal_monkey = io.imread(toDraw + 'Normal.png')
original_monkey = io.imread(toDraw + 'Original.png')

fig=plt.figure(figsize=(20, 20))

fig.add_subplot(2, 2, 1)
plt.imshow(ao_monkey)
fig.add_subplot(2, 2, 2)
plt.imshow(uv_monkey)
fig.add_subplot(2, 2, 3)
plt.imshow(normal_monkey)
fig.add_subplot(2, 2, 4)
plt.imshow(original_monkey)
fig.tight_layout()

In [ ]:
edge_monkey = filters.sobel(ao_monkey[:,:,0])
coords = feature.corner_peaks(feature.corner_fast(edge_monkey), min_distance=5)
print(coords)
plt.figure()
plt.plot(coords[:,1],coords[:,0],"ro")
plt.imshow(edge_monkey)

In [ ]:
def center_angle(p1, p2):
    x_difference = p2[1]-p1[1]
    y_difference = p2[0]-p1[0]
    return np.rad2deg(np.arctan2(y_difference, x_difference))

In [ ]:
# mean_x = np.mean(coords[1,:])
# mean_y = np.mean(coords[0,:])
# mean_point = [mean_y, mean_x]

# start = coords[np.where(spatial.distance.cdist([[0, 0]], coords).max())]
# angles = []
# for i in coords:
#     angles.append(center_angle(i, mean_point))

# angles = np.asarray(angles)
# coords = coords[angles.argsort()]
# print(start, coords)
# plt.figure()
# plt.plot(coords[:,1],coords[:,0],)
# plt.imshow(edge_monkey)

In [ ]:
dwg = svgwrite.Drawing('stroke.svg')

dwg.add(dwg.rect(insert=(0, 0), size=('100%', '100%'), rx=None, ry=None, fill='white'))

for i in range(0,10):
    genid = chr(i+48)
    gradline = dwg.linearGradient((0, 0), (1, 0), id=genid)
    gradline.add_stop_color(1, 'white', 0)
    gradline.add_stop_color(round(random.uniform(0.85, 0.95), 2), 'black')

    dwg.defs.add(gradline)
    
    start_x = random.randint(10, 100)
    start_y = random.randint(10, 100)
    
    dwg.add(dwg.line((start_x, start_y),
                     (start_x + random.randint(25, 100), start_y + random.randint(25, 100)),
                     stroke="url(#" + genid + ")",
                     stroke_linecap="round"))

display(SVG(dwg.tostring().replace('><','>\n<')))

dwg.save()

In [ ]:
def sketch(points, name):
    precision = 5
    dwg = svgwrite.Drawing(name)
    
    dwg.add(dwg.rect(insert=(0, 0), size=('100%', '100%'), rx=None, ry=None, fill='white'))
    
    gradline = dwg.linearGradient((0, 0), (1, 0), id="grad")
    #gradline.add_stop_color(1, 'white', 0.1)
    gradline.add_stop_color(1, 'black')
    
    dwg.defs.add(gradline)
    
    for i in range(1, len(points)):
        dwg.add(dwg.line((round(points[i-1][0], precision),(round(points[i-1][1], precision))),
                         ((round(points[i][0], precision), (round(points[i][1], precision)))),
                         stroke="url(#grad)",
                         stroke_linecap="round"))

    display(SVG(dwg.tostring().replace('><','>\n<')))

    return dwg

In [ ]:
edge_monkey = filters.sobel(ao_monkey[:,:,0] + normal_monkey[:,:,0])
new_edge_monkey =np.flip(np.rot90(edge_monkey), axis=0)
contours = measure.find_contours(new_edge_monkey, 0.17, fully_connected="high")
simplified_contours = [measure.approximate_polygon(c, tolerance=0) for c in contours]
plt.figure(figsize=(6,6))
for n, contour in enumerate(simplified_contours):
    plt.plot(contour[:, 0], contour[:, 1], linewidth=2, color="black")

In [ ]:
def path_sketches(points):
    precision = 5
    dwg = svgwrite.Drawing('path1.svg')

    gradline = dwg.linearGradient((0, 0), (1, 0), id="grad")
    gradline.add_stop_color(1, 'white', 0.8)
    gradline.add_stop_color(0.9, 'black')
    
    dwg.defs.add(gradline)
    
    path = dwg.path(('M', points[0][0,0], points[0][0,1]), fill='black', stroke='black')
    for pointSet in points:
        path.push('M', pointSet[0,0], pointSet[0,1])
        for p in pointSet[1:]:
            path.push(('L', p[0], p[1]))
    
    dwg.add(path)
    
    return dwg

In [ ]:
path_out = path_sketches(simplified_contours)
path_out.save()

In [ ]:
def angle(p1):
    ang = np.arctan2(p1[0], p1[1])
    return np.rad2deg((ang) % (2 * np.pi))

In [ ]:
def path_sketch(points, corners=[]):
    corners[:,[0,1]] = corners[:,[1,0]]
    dwg = svgwrite.Drawing('path1.svg')
    
    dwg.add(dwg.rect(insert=(0, 0), size=('100%', '100%'), rx=None, ry=None, fill='white'))

    gradline = dwg.linearGradient((0, 0), (0, 0), id="grad")
    gradline.add_stop_color(0.1, 'white')
    gradline.add_stop_color(1, 'black')
    
    dwg.defs.add(gradline)
    
    path = dwg.path(('M', points[0,0], points[0,1]), fill='none', stroke='black')
    for i in range(1, len(points)):
        path.push(('L', points[i,0], points[i,1]))
        print(points[i-1], points[i])
        if points[i] in corners:
            dwg.add(dwg.line((points[i,0],points[i,1]),
                             (points[i,0]+(points[i,0]-points[i-1,0])*2.5, points[i,1]+(points[i,1]-points[i-1,1])*2.5),
                             stroke='black',
                             stroke_width=round(random.uniform(0.6, 0.85), 2),
                             stroke_linecap='round'))
            
            dwg.add(dwg.line((points[i,0],points[i,1]),
                             (points[i,0]+(points[i,0]-points[i+1,0])*2.5, points[i,1]+(points[i,1]-points[i+1,1])*2.5),
                             stroke='black',
                             stroke_width=round(random.uniform(0.6, 0.85), 2),
                             stroke_linecap='round'))
    
    dwg.add(path)
    
    return dwg

In [ ]:
path_out = path_sketch(simplified_contours[0], feature.corner_peaks(feature.corner_fast(edge_monkey), min_distance=20))
path_out.save()

In [ ]:
edge_monkey = filters.sobel(normal_monkey[:,:,0]) + filters.sobel(ao_monkey[:,:,0])
feature_monkey = filters.sobel(normal_monkey[:,:,0]) - filters.sobel(ao_monkey[:,:,0])
bin_edge_monkey = np.where(edge_monkey<0.1, 0, 1)
bin_feature_monkey = np.where(feature_monkey<0.1, 0 ,1)
plt.figure(figsize=(20,20))
plt.imshow(ndimage.gaussian_filter(bin_feature_monkey, sigma=0.12), cmap = plt.cm.Greys)

In [ ]:
def momentum_sketch_fake(dwg, points):
    velocity = 0.1
    
    path = dwg.path(('M', points[0,0], points[0,1]), fill='none', stroke='black', stroke_linecap='round')
    for i in range(1, len(points)):
        velocity = 0.1
        currentPos = np.copy(points[i-1])
        
        angle = np.rad2deg(np.arctan2(points[i][1], points[i][0]))
        while lessThan(points[i-1], currentPos, points[i]):
            currentPos = [currentPos[0]+( velocity*np.cos(angle) ), currentPos[1]+( velocity*np.sin(angle) )]
            path.push(('L', currentPos[0], currentPos[1]))
            velocity = velocity + 0.2
            
        path.push(('M', points[i][0], points[i][1]))
        
    dwg.add(path)
        
    return dwg

def momentum_sketch(dwg, points):
    velocity_base = 0.1
    acceleration = 0.4
    
    path = dwg.path(('M', points[0,0], points[0,1]), fill='none', stroke='black', stroke_linecap='round')
    for i in range(1, len(points)):
        velocity = velocity_base
        currentPos = np.copy(points[i-1])
        
        while lessThan(points[i-1], currentPos, points[i]):
            difference_vector = points[i]-points[i-1]
            currentPos = [currentPos[0] + (difference_vector[0]*velocity), currentPos[1] + (difference_vector[1]*velocity)]
            path.push(('L', currentPos[0], currentPos[1]))
            velocity = velocity + acceleration
            
        path.push(('M', points[i][0], points[i][1]))
        
    dwg.add(path)
        
    return dwg

def normal_sketch(dwg, points):
    
    path = dwg.path(('M', points[0,0], points[0,1]), fill='none', stroke='black', stroke_linecap='round')
    for i in range(1, len(points)):
        currentPos = np.copy(points[i-1])
        
        path.push(('L', currentPos[0], currentPos[1]))
        
    dwg.add(path)
        
    return dwg

def momentum_sketch_test(dwg, points):
    velocity_base = 0.1
    acceleration = 0.4
    
    path = dwg.path(('M', points[0,0], points[0,1]), fill='none', stroke="#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]), stroke_linecap='round')
    for i in range(1, len(points)):
        velocity = velocity_base
        currentPos = np.copy(points[i-1])
        
        while lessThan(points[i-1], currentPos, points[i]):
            difference_vector = points[i]-points[i-1]
            currentPos = [currentPos[0] + (difference_vector[0]*velocity), currentPos[1] + (difference_vector[1]*velocity)]
            path.push(('L', currentPos[0], currentPos[1]))
            velocity = velocity + acceleration
            
        path.push(('M', points[i][0], points[i][1]))
        
    dwg.add(path)
        
    return dwg

def dotted_shade_sketch(dwg, points, radius=0.25):
    for point in points:
        dwg.add(dwg.circle(center=(float(point[0])+np.random.uniform(-0.5,0.5),float(point[1])+np.random.uniform(-0.5,0.5)), r=radius))
    
    return dwg

def dotted_shade_sketch_test(dwg, points, radius=0.25):
    color = "#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
    for point in points:
        dwg.add(dwg.circle(center=(float(point[0])+np.random.uniform(-0.5,0.5),float(point[1])+np.random.uniform(-0.5,0.5)), r=radius, style=color))
    
    return dwg

def guided_sketch(dwg, points, thickness=1):
    velocity_base = 0.1
    acceleration = 0.4
    turn_rate = 2
    
    path = dwg.path(('M', points[0,0], points[0,1]), fill='none', stroke="black", stroke_linecap='round', stroke_width=thickness)
    for i in range(1, len(points)):
        currentPos = np.copy(points[i-1])
        difference_vector = points[i]-points[i-1]
        
        if i < len(points)-1:
            next_difference_vector = points[i+1]-points[i]
        else:
            next_difference_vector = difference_vector
            
        angle = dotProductAngle(difference_vector, next_difference_vector)
        for j in range(1, turn_rate):
            nextPos = [(currentPos[0] + (difference_vector[0]*(j/turn_rate)) * np.cos(angle)), currentPos[1] + (difference_vector[1]*(j/turn_rate) * np.sin(angle))] 
            path.push(('L', nextPos[0], nextPos[1]))
            
    dwg.add(path)
        
    return dwg

def simplest_sketch(dwg, points, thickness=1):
    dwg.add(dwg.line((points[0][0], points[0][1]),
                     (points[1][0], points[1][1]),
                     stroke_linecap='round',
                     stroke_width=thickness))
    
    return dwg

def lessThan(startPos, currentPos, endPos):
    return np.linalg.norm(currentPos-startPos) < np.linalg.norm(endPos-startPos)

def dotProductAngle(v1, v2):
    #re-arranged dot product for theta
    theta = np.arccos( np.dot(v1,v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)) )
    if np.isnan(theta):
        return 0
    else:
        return theta

def rotMatrix(theta):
    return [[np.cos(theta), -np.sin(theta)],
            [np.sin(theta), np.cos(theta)]]

def initCanvas(name):
    dwg = svgwrite.Drawing(name)
    
    dwg.add(dwg.rect(insert=(0, 0), size=('100%', '100%'), rx=None, ry=None, fill='white'))

    gradline = dwg.linearGradient((0, 0), (0, 0), id="grad")
    gradline.add_stop_color(0.1, 'white')
    gradline.add_stop_color(1, 'black')

    dwg.defs.add(gradline)
    
    return dwg

### Walking the image approximately

* Needs to take an image
* Return the walked coords
* Reduce the number of points to walk from raw
* Capture the features of the image
* Single walk around the edge
* Encapsulate the whole image

In [ ]:
def getUnwalkedCoords(image):
    coords = np.where(image == 1)
    zipped_coords = np.dstack(coords)
    return zipped_coords.reshape(zipped_coords.shape[1], zipped_coords.shape[2])

def getNeighbours(currentPixel, centrePoint):
    neighbours = np.array([[currentPixel[0]+1, currentPixel[1]+1],
            [currentPixel[0]+1, currentPixel[1]],
            [currentPixel[0]+1, currentPixel[1]-1],
            [currentPixel[0], currentPixel[1]+1],
            [currentPixel[0], currentPixel[1]-1],
            [currentPixel[0]-1, currentPixel[1]+1],
            [currentPixel[0]-1, currentPixel[1]],
            [currentPixel[0]-1, currentPixel[1]-1],
           ])
    
    neighbours_dist = map(lambda x: np.linalg.norm(x-centrePoint), neighbours)
    neighbours = neighbours[np.argsort(np.array(list(neighbours_dist)))]
    return neighbours
    
def nextNeighbour(currentPixel, thinned_image, centrePoint):
    neighbours = getNeighbours(currentPixel, centrePoint)
    for n in neighbours:
        n = np.array(n).astype(int)
        try:
            if thinned_image[n[0], n[1]] == 1:
                return n
        except:
            next
    return np.array([-1,-1])

def walkNeighbourhood(image):
    thinned_image = morphology.thin(np.where(image<0.1, 0, 1))
    thinned_image = np.where(thinned_image<0.1, 0, 1)
    centre = getCentre(thinned_image)
    try:
        totalPixels = exposure.histogram(thinned_image, nbins=2)[0][1]
    except IndexError:
        return []
    currentPixel = np.array([-1, -1])
    walkingOrder = []
    index=-1
    while totalPixels > 0:
        
        if np.array_equal(currentPixel, np.array([-1, -1])):
            currentPixel = getUnwalkedCoords(thinned_image)[0]
            walkingOrder.append([])
            index += 1
        else:
            thinned_image[currentPixel[0], currentPixel[1]] = 0
            walkingOrder[index].append(currentPixel)
            currentPixel = nextNeighbour(currentPixel, thinned_image, centre)
            totalPixels -= 1
            
    return walkingOrder

#idea from https://www.geeksforgeeks.org/program-check-three-points-collinear/
def checkColinear(p1, p2, p3):
    return (p1[0] * (p2[1]-p3[1]) + p2[0] * (p3[1]-p1[1]) + p3[0] * (p1[1]-p2[1])) == 0

def getCentre(image):
    return np.mean(np.where(image==1), axis=1)

#Checking colinearity should probably be used here (maybe a recursive solution??)
def findMajorAnchors(pointsList):
    for feature in pointsList:
        changed = True
        #if the point set wasn't reduced then go to the next point set
        while changed:
            changed = False
            i = 0
            while True:

                if len(feature) < 3:
                    changed = False
                    break
                    
                if len(feature)-2 <= i:
                    break
                    
                if checkColinear(feature[i],feature[i+1],feature[i+2]):
                    feature.pop(i+1)
                    changed = True
                    
                i += 1
    return pointsList

def drawFeatures(name, edge_monkey, ao_monkey):
    pointToDraw = walkNeighbourhood(np.flip(np.rot90(edge_monkey), axis=0))
    canvas = initCanvas(name)
    pointToDraw = findMajorAnchors(pointToDraw)
    for feature in pointToDraw:
        canvas = guided_sketch(canvas, np.array(feature))
    
    sobel_outline = filters.sobel(ao_monkey[:,:,0])
    outline =  findMajorAnchors(walkNeighbourhood(np.flip(np.rot90(sobel_outline), axis=0)))
    canvas = guided_sketch(canvas, np.array(outline[0]), 1.5)
    
    return canvas

## Cleaning of paths from streamplot

In [ ]:
def clean_paths(paths):
    cleaned_paths = list()
    for path in paths:
        if len(cleaned_paths) == 0 or not np.array_equal(cleaned_paths[-1].vertices, path.vertices):
            if not np.allclose(path.vertices[0], path.vertices[1]):
                cleaned_paths.append(path)
    
    return cleaned_paths

def stitch_paths(paths):
    stitched_paths = list()
    for path in paths:
        if len(stitched_paths) == 0:
            stitched_paths.append(path.vertices)
        
        if np.allclose(stitched_paths[-1][-1], path.vertices[0]):
            joined = np.concatenate((stitched_paths[-1], np.array([path.vertices[1]])), axis=0)
            stitched_paths[-1] = joined
        else:
            stitched_paths.append(path.vertices)
    
    return stitched_paths

## Shading Implementation

* Separate shaded area from rest of image
* Band shades into K bands
* Find the gradient of each of these areas
* Map the gradients on to the areas
* Draw to output

In [ ]:
def testShading(original_image, edge_detected, k=2, threshold=24):
    original_shade = morphology.erosion(np.copy(original_image)[:,:,0] - edge_detected)
    shaded_area = np.where(original_shade >= threshold)
    original_shade[::k] = 256
    shade = np.where(original_shade < threshold, 0, 256)
    shade[shaded_area] = 0
    return shade

def getShadedArea(original_image, edge_detected, ao_im, threshold=24):
    original_shade = morphology.erosion(np.copy(original_image)[:,:,0] - edge_detected)
    shade = np.where(original_shade < threshold, 0, 256)
    ao_image = np.where(ao_im[:,:,0]>0)
    shaded_area = ao_monkey[:,:,0]-shade
    shaded_area = np.where(shaded_area > 0, 256, 0)
    return shaded_area

def getAllShadeBands(original_image, ao_image, edge_detected, bands=3, interval=30):
    previous_shade = np.zeros_like(edge_detected)
    shade_bands = []
    for band in range(1, bands+1):
        shade = getShadedArea(original_image, edge_detected, ao_image, band*interval) - previous_shade
        shade_bands.append(shade)
        previous_shade = previous_shade + shade
    
    return shade_bands
        
def applyHorizontalEffect(shaded_area, edge_detected, k=2):
    mask = np.zeros_like(edge_detected)
    mask[::k] = 256
    shaded_area = shaded_area+mask-256
    shaded_area = np.where(shaded_area > 0, 256, 0)
    return shaded_area

def applyDiagonalEffect(shaded_area, edge_detected, k=2):
    mask = np.zeros_like(edge_detected)
    changed_row = mask[0]
    changed_row[::k] = 256
    for i in range(0, len(mask)):
        mask[i] = changed_row
        changed_row = np.roll(changed_row, -1)
    shaded_area = shaded_area+mask-256
    shaded_area = np.where(shaded_area > 0, 256, 0)
    return shaded_area

def applyDottedEffect(shaded_area, edge_detected, k=0):
    mask = np.zeros_like(edge_detected)
    mask = util.random_noise(mask, mode='gaussian', mean=0.01)
    mask = np.where(mask > 0.07*k, 256, 0)
    shaded_area = shaded_area+mask-256
    shaded_area = np.where(shaded_area > 0, 256, 0)
    return shaded_area

def applyStreamEffect(shaded_area, component, k=2):
    
    filtered_uv = np.copy(component.astype(np.float64))
    filtered_uv[np.where(shaded_area==0)] = 0
    component_grad_X, component_grad_Y = np.gradient(makeContinuous(filtered_uv))

    component_grad_X = component_grad_X
    component_grad_Y = component_grad_Y

    w, h = component_grad_X.shape
    y, x = np.mgrid[0:w, 0:h]
    
    fig, ax = plt.subplots()
    ax.imshow(uv_monkey, cmap=plt.cm.Greys)
    stream = plt.streamplot(x, y, component_grad_X, component_grad_Y, density=25, linewidth=0.5, arrowsize=0.00000000001)
    fig.canvas.draw()
    lines = stitch_paths(clean_paths(stream.lines.get_paths()))
    
    return lines

def makeContinuous(component):
    component = component/np.max(component)
    component = np.sin(component*2*np.pi)
    
    return component

def findShadeAnchors(pointsList):
    newPointsList = findMajorAnchors(pointsList[:])
    for feature in pointsList:
        newPointsList.append([np.max(feature, axis=0), np.min(feature, axis=0)])
        
    return newPointsList

def drawShadeGreyBackground(canvas, original_monkey, ao_monkey, edge_monkey, bands=3, interval=30):
    currentShade = np.zeros_like(edge_monkey)
    for i in range(1, bands+1):
        shade = testShading(original_monkey, edge_monkey, 2**i, i*interval) - currentShade
        currentShade = currentShade + shade
        for feature in findShadeAnchors(walkNeighbourhood(np.flip(np.rot90(shade), axis=0))):
            canvas = guided_sketch(canvas, np.array(feature), 0.5)
    
    return canvas

def drawShadeHorizontal(canvas, original_image, ao_image, edge_detected, bands=3, interval=30):
    current_band = 1
    for shade in getAllShadeBands(original_image, ao_image, edge_detected, bands, interval):
        shade = applyHorizontalEffect(shade, edge_detected, 2**current_band)
        current_band += 1
        for feature in findShadeAnchors(walkNeighbourhood(np.flip(np.rot90(shade), axis=0))):
            canvas = guided_sketch(canvas, np.array(feature), 0.5)
    return canvas

def drawShadeDiagonal(canvas, original_image, ao_image, edge_detected, bands=3, interval=30):
    current_band = 2
    for shade in getAllShadeBands(original_image, ao_image, edge_detected, bands, interval):
        shade = applyDiagonalEffect(shade, edge_detected, 2**current_band)
        current_band += 1
        for feature in findShadeAnchors(walkNeighbourhood(np.flip(np.rot90(shade), axis=0))):
            #needed to get rid of noise
            if len(feature) > 2:
                canvas = guided_sketch(canvas, np.array(feature), 0.5)

    return canvas

def drawShadeDotted(canvas, original_image, ao_image, edge_detected, bands=3, interval=30):
    current_band = 1
    dot_radius = 0.5
    for shade in getAllShadeBands(original_image, ao_image, edge_detected, bands, interval):
        shade = applyDottedEffect(shade, edge_detected, current_band-1)
        current_band += 1
        for feature in walkNeighbourhood(np.flip(np.rot90(shade), axis=0)):
            canvas = dotted_shade_sketch(canvas, np.array(feature), dot_radius)
    
    return canvas

def drawShadeStream(canvas, original_image, ao_image, uv_component, edge_detected, bands=3, interval=30):
    current_band = 1
    
    for shade in getAllShadeBands(original_image, ao_image, edge_detected, bands, interval):
        lines = applyStreamEffect(shade, uv_component, current_band)
        current_band += 1
        lines = list(map(lambda x: x[::2].tolist(), lines))
        lines = findMajorAnchors(lines)
        if type(lines) is list:
            for feature in lines:
                feature = np.array(feature)
                if len(feature) > 5 and np.linalg.norm(feature[0]-feature[-1]) > 10:
                    guided_sketch(canvas, np.array(feature), 0.5)
        else:
            next
            
    return canvas

def drawShadeStreamV2(canvas, original_image, ao_image, uv_component, edge_detected, bands=3, interval=30):
    current_band = 1
    shades = getAllShadeBands(original_image, ao_image, edge_detected, bands, interval)
    contours = applyStreamEffect(np.ones_like(uv_component), uv_component, 1)
    
    for shade in shades:
        segmented_contours = shadeSegmentedContours(shade, contours)[::current_band]
        for feature in segmented_contours:
            feature = np.array(feature)
            if len(feature) > 3 and np.linalg.norm(feature[0]-feature[-1]) > 5:
                guided_sketch(canvas, np.array(feature), 0.5)
        current_band += 1
        
    return canvas

def shadeSegmentedContours(shade, contours):
    new_contours = list()
    prev_in_shade = False
    in_shade = False
    for contour in contours:
        
        for point in contour:
            if shade[int(point[1]), int(point[0])] == 256.0:
                in_shade = True
            else:
                in_shade = False
            
            if not prev_in_shade and in_shade:
                new_contour = list()
                new_contour.append(point)
            elif prev_in_shade and in_shade:
                new_contour.append(point)
            elif prev_in_shade and not in_shade:
                new_contours.append(new_contour)
                
            prev_in_shade = in_shade
            
        if in_shade:
            new_contours.append(new_contour)
            in_shade = False
            prev_in_shade = False
        
    return new_contours
        
    
print("Processing render...")
canvas = drawFeatures("contourV1.svg", edge_monkey, ao_monkey)
#canvas = drawShadeHorizontal(canvas, original_monkey, ao_monkey, edge_monkey, bands=3, interval=40)
#canvas = drawShadeStreamV2(canvas, original_monkey, ao_monkey, uv_monkey[:,:,0], edge_monkey, bands=3, interval=40)
canvas = drawShadeStreamV2(canvas, original_monkey, ao_monkey, uv_monkey[:,:,1], edge_monkey, bands=3, interval=40)
print("Writing " + str(len(canvas.elements)) + " paths to file...")
canvas.save()
print("Done!")

In [ ]:
# path_out = initCanvas('momentum.svg')
# path_out = momentum_sketch(path_out, simplified_contours[0])
# path_out.save()